In [1]:
import cv2
import numpy as np
import os
from pathlib import Path
import matplotlib.pyplot as plt
from abc import ABC, abstractmethod
from typing import Tuple, Any, Optional, List

In [2]:
input_dir = "../data/sofas/raw"
output_dir = "../data/sofas/processed"

In [3]:
class ImagePreprocessor(ABC):
    """Abstract base class for image preprocessing algorithms."""
    
    @abstractmethod
    def preprocess(self, image_path: str, save_path: Optional[str] = None) -> np.ndarray:
        """
        Preprocess the input image.
        
        Args:
            image_path (str): Path to the input image
            save_path (str, optional): Path to save the processed image
            
        Returns:
            np.ndarray: Processed image
        """
        pass

In [4]:
def display_all_processed_images(input_dir, output_dir):
    """
    Display each pair of original and processed images in separate figures.
    
    Args:
        input_dir (str): Directory containing original images
        output_dir (str): Directory containing processed images
    """
    # Get list of processed images
    processed_files = sorted([f for f in os.listdir(output_dir) if f.startswith('processed_')])
    original_files = sorted([f.replace('processed_', '') for f in processed_files])
    
    # Plot each pair of images in a separate figure
    for orig_file, proc_file in zip(original_files, processed_files):
        # Original image
        orig_path = os.path.join(input_dir, orig_file)
        orig_img = cv2.imread(orig_path)
        orig_img = cv2.cvtColor(orig_img, cv2.COLOR_BGR2RGB)
        
        # Processed image
        proc_path = os.path.join(output_dir, proc_file)
        proc_img = cv2.imread(proc_path)
        proc_img = cv2.cvtColor(proc_img, cv2.COLOR_BGR2RGB)
        
        # Create a new figure for each pair
        plt.figure(figsize=(20, 10))
        
        # Add original image subplot
        plt.subplot(1, 2, 1)
        plt.imshow(orig_img)
        plt.title(f'Original: {orig_file}')
        plt.axis('off')
        
        # Add processed image subplot
        plt.subplot(1, 2, 2)
        plt.imshow(proc_img)
        plt.title(f'Processed: {proc_file}')
        plt.axis('off')
        
        plt.tight_layout()
        plt.show()

In [5]:
def process_all_sofas(input_dir: str, output_dir: str, preprocessor: ImagePreprocessor):
    """
    Process all sofa images in a directory and save the processed versions.
    
    Args:
        input_dir (str): Directory containing original sofa images
        output_dir (str): Directory to save processed images
        preprocessor (ImagePreprocessor): Instance of a preprocessor class that implements ImagePreprocessor
    """
    # Create output directory if it doesn't exist
    os.makedirs(output_dir, exist_ok=True)
    
    # Process each image
    for img_file in os.listdir(input_dir):
        if img_file.lower().endswith(('.png', '.jpg', '.jpeg')):
            input_path = os.path.join(input_dir, img_file)
            output_path = os.path.join(output_dir, f'processed_{img_file}')
            
            # Process and save using the provided preprocessor
            preprocessor.preprocess(input_path, output_path)
            print(f"Processed: {img_file}")

# GrabCut-based Sofa Segmenter

## Overview
The `SofaSegmenter` class implements an approach to segment sofas from images using OpenCV's GrabCut algorithm. It's specifically optimized for sofa images, assuming they typically occupy the central portion of the image.

## Key Components

### 1. Image Preprocessing
- Resizes large images while maintaining aspect ratio
- Maximum dimension configurable (default: 800px)
- Adds padding around segmented region

### 2. Segmentation Process
1. **Initial Mask Creation**
   - Divides image into regions:
     - Border (5% of image): Definite background
     - Outer region (20-80% height, 10-90% width): Probable foreground
     - Inner region (30-70% height, 20-80% width): Definite foreground

2. **GrabCut Segmentation**
   - Applies OpenCV's GrabCut algorithm
   - Uses initial mask to guide segmentation
   - Configurable number of iterations

3. **Post-processing**
   - Extracts largest contour
   - Calculates bounding box
   - Crops to sofa region with padding
   - Scales back to original dimensions if needed

## Usage Parameters
- `padding`: Extra space around segmented sofa (default: 10px)
- `max_size`: Maximum image dimension (default: 800px)
- `iterations`: GrabCut iteration count (default: 1)

## Input/Output
- Input: RGB/BGR image file
- Output: Segmented and cropped sofa image with background removed

Based on the requirement of not using machine learning models, I think using GrabCut is a good option.

In [6]:
class SofaSegmenter(ImagePreprocessor):
    """Sofa segmentation and background removal using GrabCut algorithm."""
    
    def __init__(self, padding: int = 10, max_size: int = 800, iterations: int = 1):
        """
        Initialize the sofa segmenter.
        
        Args:
            padding (int): Padding to add around the segmented sofa
            max_size (int): Maximum dimension size for resizing while maintaining aspect ratio
            iterations (int): Number of GrabCut iterations
        """
        if padding < 0 or max_size <= 0 or iterations <= 0:
            raise ValueError("Invalid parameters: padding must be >= 0, max_size and iterations must be > 0")
        
        self.padding = padding
        self.max_size = max_size
        self.iterations = iterations
        
    def _resize_image(self, image: np.ndarray) -> Tuple[np.ndarray, float]:
        """
        Resize image while maintaining aspect ratio if it exceeds max_size.
        
        Args:
            image: Input image
            
        Returns:
            Tuple[np.ndarray, float]: (Resized image, scale factor)
        """
        height, width = image.shape[:2]
        max_dim = max(height, width)
        
        if max_dim > self.max_size:
            scale = self.max_size / max_dim
            new_width = int(width * scale)
            new_height = int(height * scale)
            resized = cv2.resize(image, (new_width, new_height), interpolation=cv2.INTER_AREA)
            return resized, scale
        return image, 1.0

    def _create_initial_mask(self, height: int, width: int) -> np.ndarray:
        """
        Create initial mask for GrabCut with sofa-specific regions.
        
        Args:
            height (int): Image height
            width (int): Image width
            
        Returns:
            np.ndarray: Initialized mask with background/foreground regions
        """
        # Initialize as probable background
        mask = np.ones((height, width), np.uint8) * cv2.GC_PR_BGD
        
        # Border parameters
        border = int(min(height, width) * 0.05)
        
        # sofa typically occupies the central portion of the image
        sofa_regions = {
            'outer': {'y': (0.2, 0.8), 'x': (0.1, 0.9), 'value': cv2.GC_PR_FGD},
            'inner': {'y': (0.3, 0.7), 'x': (0.2, 0.8), 'value': cv2.GC_FGD}
        }
        
        # Mark borders as definite background
        mask[:border, :] = cv2.GC_BGD
        mask[-border:, :] = cv2.GC_BGD
        mask[:, :border] = cv2.GC_BGD
        mask[:, -border:] = cv2.GC_BGD
        
        # Mark sofa regions
        for region in sofa_regions.values():
            y_start = int(height * region['y'][0])
            y_end = int(height * region['y'][1])
            x_start = int(width * region['x'][0])
            x_end = int(width * region['x'][1])
            mask[y_start:y_end, x_start:x_end] = region['value']
        
        return mask

    def _get_bounding_box(self, mask: np.ndarray) -> Tuple[int, int, int, int]:
        """
        Get bounding box coordinates from the largest contour in the mask.
        
        Args:
            mask: Binary mask
            
        Returns:
            Tuple[int, int, int, int]: (x, y, width, height) of bounding box
        """
        contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
        if contours:
            largest_contour = max(contours, key=cv2.contourArea)
            return cv2.boundingRect(largest_contour)
        return (0, 0, mask.shape[1], mask.shape[0])

    def _segment_sofa(self, image: np.ndarray) -> Tuple[np.ndarray, Tuple[int, int, int, int]]:
        """
        Segment sofa from image using GrabCut algorithm.
        
        Args:
            image: Input BGR image
            
        Returns:
            Tuple[np.ndarray, Tuple[int, int, int, int]]: (Segmented image, bounding box)
        """
        height, width = image.shape[:2]
        mask = self._create_initial_mask(height, width)
        
        background_model = np.zeros((1, 65), np.float64)
        foreground_model = np.zeros((1, 65), np.float64)
        
        # Perform GrabCut segmentation
        cv2.grabCut(image, mask, None, background_model, foreground_model, 
                   self.iterations, cv2.GC_INIT_WITH_MASK)
        
        # Create binary mask and apply it
        binary_mask = np.where((mask == 2) | (mask == 0), 0, 1).astype('uint8')
        segmented_image = cv2.bitwise_and(image, image, mask=binary_mask)
        
        return segmented_image, self._get_bounding_box(binary_mask)

    def preprocess(self, image_path: str, save_path: Optional[str] = None) -> np.ndarray:
        """
        Preprocess sofa image using GrabCut-based segmentation.
        
        Args:
            image_path: Path to input image
            save_path: Optional path to save the processed image
            
        Returns:
            np.ndarray: Processed image with background removed and cropped
        """
        # Load and validate image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Failed to load image from {image_path}")
        
        # Process image at reduced size for efficiency
        resized_image, scale = self._resize_image(image)
        segmented_image, (x, y, w, h) = self._segment_sofa(resized_image)
        
        # Scale coordinates back to original size if needed
        if scale != 1.0:
            x, y, w, h = [int(val / scale) for val in (x, y, w, h)]
            segmented_image = cv2.resize(segmented_image, (image.shape[1], image.shape[0]), 
                                       interpolation=cv2.INTER_CUBIC)
        
        # Add padding and ensure coordinates are within image bounds
        x = max(0, x - self.padding)
        y = max(0, y - self.padding)
        w = min(image.shape[1] - x, w + 2 * self.padding)
        h = min(image.shape[0] - y, h + 2 * self.padding)
        
        # Crop to sofa region
        result = segmented_image[y:y+h, x:x+w]
        
        if save_path:
            cv2.imwrite(save_path, result)
        
        return result

In [ ]:
preprocessor = SofaSegmenter(
    padding=20,
    max_size=800
)

process_all_sofas(input_dir, output_dir, preprocessor)
display_all_processed_images(input_dir, output_dir)